# Visualizing a Network Graph
### Author: Nathalie A. Vladis PhD

To execute this Notebook seamlessly you need to have a csv file called 'Relations' representing collaborator relationships and a csv file called 'Groups' listing each collaborator's affiliation.

In [ ]:
# In this cell we import all required packages
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx

In [ ]:
# Here we specify the colors we want for each affiliation goup
group_cols = {'Group 1': '#1f5da2',
              'Group 2': '#53b129',
              'Group 3': '#48a4e3', 
              'Group 4': '#f59e28', 
             }

Here are some additional colour-blind friendly options to choose from.
<a href="https://imgur.com/bkpsxfL"><img src="https://i.imgur.com/bkpsxfL.png" title="source: imgur.com" /></a>

In [ ]:
# Importing datasets as specified in the introduction
relationships = pd.read_csv('Relations.csv')
groups = pd.read_csv('Groups.csv')

In [ ]:
# Checking that our relationships look as we expected
relationships.head()

In [ ]:
# Ensuring that we have the groups we expected (no missing and no extras)
groups.Group.unique()

In [ ]:
# Ensuring that we have the expect number of members for each group
from collections import Counter
group_counts = Counter(groups.Group)
print(group_counts)

In [ ]:
# The code in this cell specifies parameters and creates the graph visualization
# NB: For esthetic purposes 'row_limit' helps us specify how many collaborators we want to see per row
group_names = [x for x in group_counts]
row_limit = 5
subset_sizes = []
subset_names = []
subset_color = []

for group in group_names:
    group_members = [x for x in groups[groups.Group == group].Node.unique()]
    count = group_counts[group]
    color = group_cols[group]
    
    while count > 0:
        if count > row_limit:
            subset_sizes.append(row_limit)
            subset_color.append(color)
            subset_names.append(group_members[:row_limit])
            group_members = group_members[row_limit:]
            count -= row_limit
        else:
            subset_sizes.append(count)
            subset_color.append(color)
            subset_names.append(group_members)
            count = 0
                


def multilayered_graph(layers):
    G = nx.Graph()
    for (i, layer) in enumerate(layers):
        G.add_nodes_from(layer, layer=i)
    for line in relationships.iterrows():
        a, b = line[1].Node.strip(), line[1].Relationship.strip()
        
        if relationships.Node.str.contains(a).any() and relationships.Node.str.contains(b).any():
            
            G.add_edge(a, b, alpha=0.3)
        
    return G



G = multilayered_graph(subset_names)
color = [subset_color[data["layer"]] for v, data in G.nodes(data=True)]
pos = nx.multipartite_layout(G, subset_key="layer", scale=20)
plt.figure(figsize=(20,8), dpi=300)
# nx.draw(G, pos, node_color=color, with_labels=True)
nx.draw_networkx_edges(G, pos, alpha=0.2, width=0.8)
nx.draw_networkx_nodes(G, pos, node_color=color, node_size=250, alpha=0.9)


for l in pos:
    pos[l][1] += 0.5

# Argument font_family chooses fonts for labels, popular optons include serif & sans-serif

nx.draw_networkx_labels(G, pos, verticalalignment='baseline', horizontalalignment='center', font_family='sans-serif')
plt.axis("equal")
plt.show()

## Enjoy your new network visualization!